**Data Loading**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
chatbot = pd.read_csv('chatbot_cleaned.csv')
print(chatbot.shape)
chatbot.head()

(80, 4)


,response_id,class,response_text,text_token
0,response_1,0,try avoid sort conflict,"['try', 'avoid', 'sort', 'conflict']"
1,response_2,1,friend open mental addiction weed taking life ...,"['friend', 'open', 'mental', 'addiction', 'wee..."
2,response_3,1,saved girl suicide going swallow bunch pills t...,"['saved', 'girl', 'suicide', 'going', 'swallow..."
3,response_4,0,cant think one really think may indirectly,"['cant', 'think', 'one', 'really', 'think', 'm..."
4,response_5,0,really one friend fit categories therapist cal...,"['really', 'one', 'friend', 'fit', 'categories..."


In [3]:
resume = pd.read_csv('resume_cleaned.csv')
print(resume.shape)
resume.head()

(125, 4)


,resume_id,class,resume_text,text_token
0,resume_1,0,customer service supervisor tier isabella cata...,"['customer', 'service', 'supervisor', 'tier', ..."
1,resume_2,0,engineer scientist ibm microelectronics divisi...,"['engineer', 'scientist', 'ibm', 'microelectro..."
2,resume_3,0,lts software engineer computational lithograph...,"['lts', 'software', 'engineer', 'computational..."
3,resume_4,0,tutor williston email indeed indeed com alec s...,"['tutor', 'williston', 'vt', 'email', 'indeed'..."
4,resume_5,1,independent consultant self employed burlingto...,"['independent', 'consultant', 'self', 'employe..."


**Encoding**

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
ohe = preprocessing.OneHotEncoder()
tfidf = TfidfVectorizer()
countvec = CountVectorizer()

In [5]:
chatbot_X = tfidf.fit_transform(chatbot['response_text'])
chatbot_y = chatbot['class']

resume_X = tfidf.fit_transform(resume['resume_text'])
resume_y = resume['class']

**Train/Test Splitting**

In [6]:
from sklearn.model_selection import train_test_split

chatbot_X_train, chatbot_X_test, chatbot_y_train, chatbot_y_test = train_test_split(chatbot_X, chatbot_y, test_size=0.2, random_state=42)
resume_X_train, resume_X_test, resume_y_train, resume_y_test = train_test_split(resume_X, resume_y, test_size=0.2, random_state=42)

chatbot_X_train = chatbot_X_train.toarray()
chatbot_X_test = chatbot_X_test.toarray()
resume_X_train = resume_X_train.toarray()
resume_X_test = resume_X_test.toarray()


In [7]:
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, SimpleRNN, Embedding

2024-04-21 06:21:00.840655: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 06:21:00.865007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 06:21:00.865584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 06:21:01.428826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


**Modeling**

*RNN*

Simple RNN Model (No Dropout)

In [8]:
simple_rnn_model = Sequential()
simple_rnn_model.add(Embedding(1000, 32))
simple_rnn_model.add(SimpleRNN(100))
simple_rnn_model.add(Dense(1, activation='sigmoid'))

simple_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

simple_rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               13300     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 45401 (177.35 KB)
Trainable params: 45401 (177.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-04-21 06:21:01.878622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-21 06:21:01.878897: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
print("Chatbot Simple RNN Accuracy Results:")
simple_rnn_model.fit(chatbot_X_train, chatbot_y_train, epochs=5, batch_size=32, validation_data=(chatbot_X_test, chatbot_y_test))

Chatbot Simple RNN Accuracy Results:
Epoch 1/5


2/2 [==============================] - 1s 201ms/step - loss: 0.6651 - accuracy: 0.6406 - val_loss: 0.4421 - val_accuracy: 0.8750
Epoch 2/5
2/2 [==============================] - 0s 64ms/step - loss: 0.6645 - accuracy: 0.6406 - val_loss: 0.5223 - val_accuracy: 0.8750
Epoch 3/5
2/2 [==============================] - 0s 66ms/step - loss: 0.6590 - accuracy: 0.6406 - val_loss: 0.5840 - val_accuracy: 0.8750
Epoch 4/5
2/2 [==============================] - 0s 74ms/step - loss: 0.6606 - accuracy: 0.6406 - val_loss: 0.5326 - val_accuracy: 0.8750
Epoch 5/5
2/2 [==============================] - 0s 65ms/step - loss: 0.6565 - accuracy: 0.6406 - val_loss: 0.4987 - val_accuracy: 0.8750


In [10]:
print("Resume Simple RNN Accuracy Results:")
simple_rnn_model.fit(resume_X_train, resume_y_train, epochs=5, batch_size=32, validation_data=(resume_X_test, resume_y_test))

Resume Simple RNN Accuracy Results:
Epoch 1/5
4/4 [==============================] - 18s 4s/step - loss: 0.5560 - accuracy: 0.7700 - val_loss: 0.7613 - val_accuracy: 0.6000
Epoch 2/5
4/4 [==============================] - 16s 4s/step - loss: 0.5413 - accuracy: 0.7700 - val_loss: 0.8272 - val_accuracy: 0.6000
Epoch 3/5
4/4 [==============================] - 16s 4s/step - loss: 0.5576 - accuracy: 0.7700 - val_loss: 0.8092 - val_accuracy: 0.6000
Epoch 4/5
4/4 [==============================] - 16s 4s/step - loss: 0.5352 - accuracy: 0.7700 - val_loss: 0.7077 - val_accuracy: 0.6000
Epoch 5/5
4/4 [==============================] - 16s 4s/step - loss: 0.5645 - accuracy: 0.7700 - val_loss: 0.6754 - val_accuracy: 0.6000


RNN with Dropout

In [11]:
dropout_rnn_model = Sequential()
dropout_rnn_model.add(Embedding(1000, 32))
dropout_rnn_model.add(SimpleRNN(100, dropout=0.2, recurrent_dropout=0.2))
dropout_rnn_model.add(Dense(1, activation='sigmoid'))

dropout_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

dropout_rnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          32000     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               13300     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 45401 (177.35 KB)
Trainable params: 45401 (177.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
print("Chatbot Dropout RNN Accuracy Results:")
dropout_rnn_model.fit(chatbot_X_train, chatbot_y_train, epochs=5, batch_size=32, validation_data=(chatbot_X_test, chatbot_y_test))

Chatbot Dropout RNN Accuracy Results:
Epoch 1/5


2/2 [==============================] - 1s 159ms/step - loss: 0.6268 - accuracy: 0.6406 - val_loss: 0.7732 - val_accuracy: 0.1250
Epoch 2/5
2/2 [==============================] - 0s 62ms/step - loss: 0.7151 - accuracy: 0.5312 - val_loss: 0.8168 - val_accuracy: 0.1250
Epoch 3/5
2/2 [==============================] - 0s 61ms/step - loss: 0.7151 - accuracy: 0.5781 - val_loss: 0.8213 - val_accuracy: 0.1250
Epoch 4/5
2/2 [==============================] - 0s 62ms/step - loss: 0.7158 - accuracy: 0.5156 - val_loss: 0.8146 - val_accuracy: 0.1250
Epoch 5/5
2/2 [==============================] - 0s 64ms/step - loss: 0.7099 - accuracy: 0.5156 - val_loss: 0.7950 - val_accuracy: 0.1250


In [13]:
print("Resume Dropout RNN Accuracy Results:")
dropout_rnn_model.fit(resume_X_train, resume_y_train, epochs=5, batch_size=32, validation_data=(resume_X_test, resume_y_test))

Resume Dropout RNN Accuracy Results:


Epoch 1/5
4/4 [==============================] - 20s 5s/step - loss: 0.6701 - accuracy: 0.5500 - val_loss: 0.7040 - val_accuracy: 0.4000
Epoch 2/5
4/4 [==============================] - 19s 5s/step - loss: 0.7320 - accuracy: 0.4600 - val_loss: 0.6888 - val_accuracy: 0.6000
Epoch 3/5
4/4 [==============================] - 19s 5s/step - loss: 0.6893 - accuracy: 0.5900 - val_loss: 0.6817 - val_accuracy: 0.6000
Epoch 4/5
4/4 [==============================] - 19s 5s/step - loss: 0.7436 - accuracy: 0.4900 - val_loss: 0.6775 - val_accuracy: 0.6000
Epoch 5/5
4/4 [==============================] - 20s 5s/step - loss: 0.7184 - accuracy: 0.5000 - val_loss: 0.6750 - val_accuracy: 0.6000
